In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
import cv2
from tqdm import tqdm          
from random import shuffle
from zipfile import ZipFile
import glob
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import tensorflow as tf

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os

from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader
from tflite_model_maker import model_spec
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig



C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [2]:
data = DataLoader.from_folder('./train/')

INFO:tensorflow:Load image with size: 10015, num_label: 7, labels: Bowens_disease, basal_cell_carcinoma, benign_keratoses, dermatofibroma, melanocytic_nevi, melanoma, vascular_lesions.


In [3]:
train_data, test_data = data.split(0.7)

In [4]:
from tflite_model_maker.image_classifier import Resnet50Spec
resnet_spec = Resnet50Spec(
    uri='https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4',
    compat_tf_versions=2, input_image_shape=None, name='resnet_50'
)

In [7]:
model = image_classifier.create(train_data,
                                validation_data = test_data,
                                model_spec=resnet_spec,
                                epochs = 20,
                                shuffle=True)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2_2 (Hub  (None, 2048)             23564800  
 KerasLayerV1V2)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 7)                 14343     
                                                                 
Total params: 23,579,143
Trainable params: 14,343
Non-trainable params: 23,564,800
_________________________________________________________________
None
Epoch 1/20
219/219 [==============================] - 264s 1s/step - loss: 1.5091 - accuracy: 0.6426 - val_loss: 1.3113 - val_accuracy: 0.6912
Epoch 2/20
219/219 [==============================] - 261s 1s/step - loss: 1.2646 - acc

In [8]:
model.export(export_dir='./',
             tflite_filename='resnet_20epochs.tflite', 
             quantization_config=QuantizationConfig.for_float16())

INFO:tensorflow:Assets written to: C:\Users\Tau\AppData\Local\Temp\tmps9x3j4z0\assets


INFO:tensorflow:Assets written to: C:\Users\Tau\AppData\Local\Temp\tmps9x3j4z0\assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in C:\Users\Tau\AppData\Local\Temp\tmptcs72gw7\labels.txt


INFO:tensorflow:Saving labels in C:\Users\Tau\AppData\Local\Temp\tmptcs72gw7\labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./resnet_20epochs.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./resnet_20epochs.tflite


In [9]:
loss, accuracy = model.evaluate(test_data)

94/94 [==============================] - 81s 823ms/step - loss: 1.2271 - accuracy: 0.7225
